In [1]:
# IMPORTS
import requests
import numpy as np
import time
import pandas as pd
import os
import datetime
from io import TextIOWrapper
from fastparquet import write
from pandas import DataFrame
from riotwatcher import LolWatcher, ApiError
from pynput import keyboard

In [2]:
API_KEY = 'RGAPI-edbbe360-037b-4578-852a-bec7b45309b0'
REGIONS = ['na1', 'kr', 'euw1']
RANKS = ['PLATINUM', 'DIAMOND']
TIERS = ['I', 'II', 'III', 'IV']
TASKS = [(lambda x: x, 'puuids')]
DATA_DIR = 'C://Repository//MOBA_Project//data'
lol_watcher = LolWatcher(API_KEY)

In [3]:
def write_to_parquet(task: str, df: DataFrame, region: str, rank: str, tier: str) -> str:
    DIRECTORY_PATH = f'F://data//{task}//{region}//'
    file_name = f'{task}_{rank}_{tier}.parquet'
    file_path = DIRECTORY_PATH + file_name

    # create the parquet if it doesn't exist
    if not os.path.isfile(file_path):
        print(f'Creating {file_name}')
        write(file_path, df)
    else:
        write(file_path, df, append=True)
    return file_path

In [4]:
def get_summoner_puuids(region: str, summonerID: str) -> DataFrame:
    """
    Gets random summoner ids from North America from the various ranks (Diamond:Platinum:Gold:Silver:Bronze)
    Returns a pandas dictionary only with summoner ids that have wins + losses >= 20
    Args:
        region (str): The region we want to search in
        rank (str): The rank we want to get from
        division (str): The divions we want to get from
        num_pages (int, optional): An amount of summoner ids to get, a group of 204 are a 'page'. Defaults to 4.

    Returns:
        bool: Whether getting the summoner ids was successful
    """
    max_retries = 3
    retry_delay = 2
    for attempt in range(max_retries):
        try:
            # attempt to get a response
            response = lol_watcher.summoner.by_id(region, summonerID)

            return pd.DataFrame.from_records(response, index=[0])

        except ApiError as err:
            print("Attempt", attempt + 1, "failed:", err)
            time.sleep(retry_delay)
    
    print(f'Failed at summonerID: {summonerID}')
    return None

## Step 1: Get the PUUIDs

In [5]:
def get_puuids(region: str, rank: str, tier: str, summoners: DataFrame):
    print(f'Number of {rank} {tier} summonerIds: {summoners.shape[0]}')
    seen_summoners = set()
    num_requested = 0
    out_df = pd.DataFrame()
    for summoner in summoners['summonerId']:
        num_requested += 1
        
        if summoner in seen_summoners:
            continue

        seen_summoners.add(summoner)
        df = get_summoner_puuids(region, summoner)

        if df is None:
            continue

        if out_df.shape[0] == 0:
            out_df = df.copy(deep=True)
        else:
            out_df = pd.concat([out_df, df], axis=0)

        if num_requested % 100 == 0:
            print(f'Finished: {num_requested}')

    write_to_parquet(out_df, region, rank, tier)

In [16]:
def get_task_df(task, region, rank, tier, cols) -> DataFrame:
    prev_tasks_map = {'puuids': 'summoner', 'matchIds': 'puuids'}
    prev_task = prev_tasks_map[task]

    # read the summoner dataframe for the given region
    file_name = f'//{prev_task}//{region}//{prev_task}_{rank}_{tier}.parquet'
    complete_fp = DATA_DIR + file_name

    return pd.read_parquet(complete_fp, columns=[cols])

In [17]:
summoner_df = get_task_df('puuids', 'na1', 'DIAMOND', 'II', 'summonerId')

In [18]:
summoner_df

,summonerId
0,KdgmRdGwQn1m19U8dR0rg5Y7tdtugb9B6IvzJD0V9ELGYs8
1,HoTw1lnCMetth0h3BbXuQ5mHXj7W7PwMCIkfuKlkdQ_DVd0
2,4ozTyhNWaHdo8UI3xGDz1boRFiDPN55Vm0f_wqpkFOsgC9g
3,Lmo8HGch3qjlMhXzgJyHWkWUn0hibIhco41IN3FCaDF8oeU
4,jZWVMcycWgyZeBUzVyEzI96a1V-gp4BOFJlRhVSD6Qdzx_c
...,...
3734,hk0TDr27f122MB0Ay7EnfBrVfEAjC6KJ-XDMDSs7bEcARWE
3735,YI-Pxt7X16H-QVXVUMzob2UxBguq92oUi0jUExDo8JvWlTk
3736,nKMw34mdJdq5qdOEdbiyZHcni2-V0BFI7fUzkge-K6YlRwM
3737,hL-Vvg3hpQ5SlEfrPhe-xY5C74euMSSmL637nh5Xi1u4mF8


In [24]:
get_puuids('na1', 'DIAMOND', 'II', summoner_df)

Number of DIAMOND II summonerIds: 3739
Finished: 100
Finished: 200


## Step 2: Get the Match IDs

In [19]:
def get_match_id(route: str, puuid: str) -> DataFrame:
    max_retries = 3
    retry_delay = 2
    for attempt in range(max_retries):
        try:
            # attempt to get a response
            response = lol_watcher.match.matchlist_by_puuid(route, puuid, queue=700, count=100)
            if len(response) == 0:
                return None

            out_df = pd.DataFrame.from_dict({'match_ids': response})
            out_df['puuid'] = puuid
            return out_df

        except ApiError as err:
            print("Attempt", attempt + 1, "failed:", err)
            time.sleep(retry_delay)
    
    print(f'Failed at puuid: {puuid}')
    return None

In [20]:
puuids_df = get_task_df('matchIds', 'na1', 'DIAMOND', 'I', 'puuid')

In [21]:
puuids_df

,puuid
index,
0,bmQ_Q7xWpOM9sD0HkFODWy5qAFShLoGS5V2utYqiBZ5jv8...
0,ta7pflkBOweD0usqFX4xFjGgK1xcArgwWRO8B-7C8O4x8O...
0,9NP1wvNEn2QYtfkrjl7Td40PAMv1v3eoAMStYP_561lDD7...
0,fwKYSlyH3J5FHHCwzjyV7AhC3wDtpgP6uPCs8hszvZx_8p...
0,2Ww9m4cRWQ828Sg331LOxHPMcLm2mPeET8sOhx7k27Pm13...
...,...
0,Dtc2H2KIM4xpoemBM3TIq5SJgV3cCHKhTFpn0_hL78vGMI...
0,1OZTvjmafRoTqfu6WVi_GOz-cLDoqUC8QFvJ95d2lBZ94j...
0,7BpNCMmB1u4e3Wazef1dwbbLaGVRQhR2dppJ58Ochg7md2...


In [ ]:
def batch_match_ids(route: str, region: str, rank:str, tier:str, puuids: DataFrame)-> DataFrame:
    print(f'Number of {rank} {tier} piuuids: {puuids.shape[0]}')
    seen_summoners = set()
    num_requested = 0
    out_df = pd.DataFrame()
    for summoner in summoners['summonerId']:
        num_requested += 1
        
        if summoner in seen_summoners:
            continue

        seen_summoners.add(summoner)
        df = get_summoner_puuids(region, summoner)

        if df is None:
            continue

        if out_df.shape[0] == 0:
            out_df = df.copy(deep=True)
        else:
            out_df = pd.concat([out_df, df], axis=0)

        if num_requested % 100 == 0:
            print(f'Finished: {num_requested}')

    write_to_parquet(out_df, region, rank, tier)

In [11]:
get_match_id('AMERICAS', 'AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf15UUuUcts9M63UdwaAhywgzwBpHhZCoQ')

,match_ids,puuid
0,NA1_4650069425,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
1,NA1_4650010291,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
2,NA1_4649959815,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
3,NA1_4649122464,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
4,NA1_4649070878,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
5,NA1_4649009287,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
6,NA1_4637118754,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
7,NA1_4637059486,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
8,NA1_4637012067,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...
9,NA1_4625879090,AiJfCXEn25HtH6oQZCkokvQ-GF3dJZH3zG_t5rTk_8YVTf...


In [ ]:
for region in REGIONS:
    for task, taskname in TASKS:
        for rank in RANKS:
            for tier in TIERS:
                if task:
                    df = get_task_df()